In [1]:
#сделать
#загрузка через таблицу файлов
#обновление и сохранение бд (id всегда уникальны, данные не удалются из папки)
#добавить выгрузку состояний опасности

In [2]:
import matplotlib.pyplot as plt

In [ ]:

import plotly.graph_objects as go
from datetime import datetime
import dash
import dash_core_components as dcc
import dash_html_components as html
import webbrowser
import dash
from dash.dependencies import Input, Output, State
import dash_table
import dash_core_components as dcc
import dash_html_components as html
import sqlite3
import pandas as pd
import numpy as np
import dash_core_components as dcc
import plotly.express as px
import matplotlib.pyplot as plt
import os
import wfdb
from scipy import signal
import dash
import dash_auth
import dash_core_components as dcc
import dash_html_components as html
import plotly
import dash_bootstrap_components as dbc
# Since we're adding callbacks to elements that don't exist in the app.layout,
# Dash will raise an exception to warn us that we might be
# doing something wrong.
# In this case, we're adding the elements through a callback, so we can ignore
# the exception.
cnx = sqlite3.connect('data/databases/patients.db')
df = pd.read_sql_query("SELECT * FROM patients", cnx)
#global df

#df = pd.read_csv('data/databases/patients.csv')
params_df_page_1 = ['ID','Имя','Фамилия','Отчество','Пол','Возраст','ЭКГ','Температура','Пульс']
params_df_page_2 = ['Дата обследования', 'ЭКГ', 'Температура', 'Пульс']


external_stylesheets=[
    "https://stackpath.bootstrapcdn.com/bootstrap/4.1.3/css/bootstrap-grid.min.css"
]

# Keep this out of source code repository - save in a file or a database
VALID_USERNAME_PASSWORD_PAIRS = {
    '0000': '0000'
}

app = dash.Dash(__name__, suppress_callback_exceptions=True, external_stylesheets=external_stylesheets)
auth = dash_auth.BasicAuth(
    app,
    VALID_USERNAME_PASSWORD_PAIRS
)

app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content'),
    dcc.Store(id='active_cell_value'),
    dcc.Store(id='database'),
    dcc.Store(id='dataframe')
])


index_page = html.Div([
    dcc.Link('Запустить приложение', href='/page-1'),
    #html.Br(),
    #dcc.Link('Go to Page 2', href='/page-2'),
])

page_1_layout = html.Div([
    
    html.Br(),
    html.Div(id='page-1-content'),
    html.Br(),
    
    
    
    dash_table.DataTable(
        id='adding-rows-table',
        data=df.to_dict('records'),
        columns=[{'id': c, 'name': c} for c in params_df_page_1],

        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="multi",
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
        
        editable=True,
        row_deletable=True
    ),

    #html.Button('Добавить пациента', id='editing-rows-button', n_clicks=0),
    #html.Button('Обновить базу данных', id='update-bd-button', n_clicks=0),
])


page_2_layout = html.Div([
    html.Div(id='page-content_2'),
    html.Br(),

    dash_table.DataTable(
        id='df_page_2',
        
        columns=[{'id': c, 'name': c} for c in params_df_page_2],

        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="multi",
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
        
        editable=True,
        row_deletable=True
    ),
    
    
    html.Br(),
    
    
    
    html.Div([
    dcc.Graph(id = 'ecg_graph'),
    html.Br(),
    dcc.Graph(id = 'pulse_graph'),
    html.Br(),
    dcc.Graph(id = 't_graph'),
    html.Br()],
        
    style={'width': '60%',
          'display': 'inline-block',
          'border': '1'}
    ),
    
    html.Div([
    html.P(id = 'beat'),
    html.Br(),       
    html.P(id = 'ecg_f_min'),
    html.Br(),
    html.P(id = 'ecg_f'),
    html.Br(),
    html.P(id = 'ecg_f_max'),
    html.Br(),
    #html.P('1', id = 'pulse_f'),
    #html.Br(),
    html.P(id = 't_f_min'),
    html.Br(),
    html.P(id = 't_f'),
    html.Br(),
    html.P(id = 't_f_max'),
    html.Br(),
    html.Br(), 
    html.P(id = 'danger_ecg'),
    html.Br(),
    #html.P('2',id = 'danger_pulse'),
    #html.Br(),
    html.P(id = 'danger_t'),
    html.Br(),
    html.Br()],
    #html.Button('Показать состояние опасности', id='danger-bd-button', n_clicks=0)],
    
    style={'width': '20%',
           "margin-top": "5%",
           "padding": "5px",
          'display': 'inline-block',
          'border': '1'}
    )
    
])




@app.callback(
    Output('url', 'pathname'),
    Output('active_cell_value', 'value'),
    Input('adding-rows-table', 'active_cell'))
def display_page(active_cell):
    #print(active_cell)
    if active_cell['column_id'] == 'ID':
        active_row_id = active_cell['row'] if active_cell else None
        print(active_row_id)
        print('active_cell', active_cell)
        return '/page-2', active_row_id
    else: None

@app.callback(
    Output('df_page_2', 'data'),
    Input('active_cell_value', 'value'))
def df2(value):
    active_row_id = value
    print(value)
    print(active_row_id)
    
    if active_row_id != None:
        try:
            data_folder = 'data/signals'
            all_patients = os.listdir(data_folder)
            patients_id = [i.split('_')[-1] for i in all_patients]
            all_dates_folder = os.listdir(data_folder + '/' + 'patients_' + patients_id[active_row_id])
            
            
            
            data=[
                dict({'Дата обследования': all_dates_folder[i], 
                      'ЭКГ':len(os.listdir(data_folder + '/' + 'patients_' + patients_id[active_row_id] + '/' + all_dates_folder[i] + '/ecg')) > 0, 
                      'Температура':len(os.listdir(data_folder + '/' + 'patients_' + patients_id[active_row_id] + '/' + all_dates_folder[i] + '/temperature')) > 0, 
                      'Пульс':len(os.listdir(data_folder + '/' + 'patients_' + patients_id[active_row_id] + '/' + all_dates_folder[i] + '/pulse')) > 0})
                for i in range(len(all_dates_folder))
            ]
        except:
            data = [dict({'Дата обследования': 'None',
                          'ЭКГ':'None',
                          'Температура':'None',
                          'Пульс':'None'})
            ]
    else: 
        data = [dict({'Дата обследования': 'None',
                      'ЭКГ':'None',
                      'Температура':'None',
                      'Пульс':'None'})
        ]
        
    return data        

@app.callback(
    Output('t_graph', 'figure'),
    Output('t_f_min', 'children'),
    Output('t_f', 'children'),
    Output('t_f_max', 'children'),
    Output('danger_t', 'children'),
    Input('active_cell_value', 'value'),
    Input('df_page_2', 'active_cell'),
    Input('df_page_2', 'data'))
def plot_t(value, active_cell, data):
    fig = go.Figure()
    active_row_id = value
    active_cell = active_cell['row'] if active_cell else None
    if active_row_id != None:
        fs = 100
        data_folder = 'data/signals'
        all_patients = os.listdir(data_folder)
        patients_id = [i.split('_')[-1] for i in all_patients]
        patient_rec = os.listdir(data_folder + '/' + 'patients_' + patients_id[active_row_id] + '/' + data[active_cell]['Дата обследования'] + '/temperature')
        all_data = pd.read_csv(data_folder + '/' + 'patients_' + patients_id[active_row_id]+ '/' + data[active_cell]['Дата обследования'] + '/temperature/' + patient_rec[0], skiprows = 9, sep = ';')
        
        data_t = all_data[all_data.columns[0]]
        data_t = data_t[:1000]
        fig.add_trace(go.Line(y = data_t/1000, x = np.array([i/fs for i in range(len(data_t))])))
    else:
        fig.add_trace(go.Line(np.array([0,0,0,0,0,0,0,0,0,0])))
    fig.update_xaxes(title_text='Температура (время, с)', range=(0, fs*2/fs))
    fig.update_yaxes(title_text='С')
    
    fig.update_layout(showlegend=False)
    pulse_dang_value_max = 39.0
    pulse_dang_value_min = 35.8
    a = np.array([i/fs for i in range(len(data_t))])[np.where((data_t/1000 > pulse_dang_value_max) | (data_t/1000 < pulse_dang_value_min))[0]]
    
    if len(a) > 0:
        fig.add_trace(go.Scatter(
            x=a, y=np.array([39 for i in a]),
            mode='markers',
            marker_color='rgba(152, 0, 0, .8)'
        ))   
        dang = 'присутствует'
    else:
        dang = 'отсутствует'
                      
    return fig, 'Минимальная температура {}'.format(round(min(data_t/1000), 2)), \
        'Средняя температура {}'.format(round(np.mean(data_t/1000), 2)), \
        'Максимальная температура {}'.format(round(np.max(data_t/1000), 2)), \
        'Состояние опасности (температура) {}'.format(dang)

@app.callback(
    Output('pulse_graph', 'figure'),
    #Output('pulse_f', 'children'),
    #Output('danger_pulse', 'children'),
    Input('active_cell_value', 'value'),
    Input('df_page_2', 'active_cell'),
    Input('df_page_2', 'data'))
def plot_pulse(value, active_cell, data):
    active_row_id = value
    active_cell = active_cell['row'] if active_cell else None
    if active_row_id != None:
        data_folder = 'data/signals'
        all_patients = os.listdir(data_folder)
        patients_id = [i.split('_')[-1] for i in all_patients]
        patient_rec = os.listdir(data_folder + '/' + 'patients_' + patients_id[active_row_id] + '/' + data[active_cell]['Дата обследования'] + '/pulse')
        record = wfdb.rdrecord(data_folder + '/' + 'patients_' + patients_id[active_row_id] + '/' + data[active_cell]['Дата обследования'] + '/pulse/' + patient_rec[1][:-4], sampto=2500) 
        
        try:
            fig = px.line(x = np.array([i/record.fs for i in range(len(record.p_signal))]), y = record.p_signal[:,0])
        except:
            fig = px.line(x = np.array([i/record.fs for i in range(len(record.p_signal))]), y = record.p_signal[:,0])
    else:
        fig = px.line(np.array([0,0,0,0,0,0,0,0,0,0]))
    fig.update_xaxes(title_text='Пульсовая волна (время, с)', range=(0, record.fs*2/record.fs))
    fig.update_yaxes(title_text='mmHg')   
    fig.update_layout(showlegend=False)
    return fig#, 'Средняя пульсовая волна "{}"'.format(np.mean(record.p_signal)), 'Состояние опасности (пульсовая волна) "{}"'.format('отсутствует')


@app.callback(
    Output('ecg_graph', 'figure'),
    Output('ecg_f_min', 'children'),
    Output('ecg_f', 'children'),
    Output('ecg_f_max', 'children'),
    Output('beat', 'children'),
    Output('danger_ecg', 'children'),
    Input('active_cell_value', 'value'),
    Input('df_page_2', 'active_cell'),
    Input('df_page_2', 'data'))
def plot_ecg(value, active_cell, data):
    active_row_id = value
    active_cell = active_cell['row'] if active_cell else None    
    fig = go.Figure()
    if active_row_id != None:
        data_folder = 'data/signals'
        all_patients = os.listdir(data_folder)
        patients_id = [i.split('_')[-1] for i in all_patients]
        patient_rec = os.listdir(data_folder + '/' + 'patients_' + patients_id[active_row_id] + '/' + data[active_cell]['Дата обследования'] + '/ecg')
        record = wfdb.rdrecord(data_folder + '/' + 'patients_' + patients_id[active_row_id] + '/' + data[active_cell]['Дата обследования'] + '/ecg/' + patient_rec[1][:-4], sampto=2500) 
        try:
            atr = wfdb.rdann(data_folder + '/' + 'patients_' + patients_id[active_row_id] + '/' + data[active_cell]['Дата обследования'] + '/ecg/' + patient_rec[1][:-4], 'ecg', sampto=2500) 
        except:
            atr = wfdb.rdann(data_folder + '/' + 'patients_' + patients_id[active_row_id] + '/' + data[active_cell]['Дата обследования'] + '/ecg/' + patient_rec[1][:-4], 'qrs', sampto=2500) 
        try:    
            fig.add_trace(go.Line(y = record.p_signal[:,1], x = np.array([i/atr.fs for i in range(len(record.p_signal[:,1]))])))
        except:
            fig.add_trace(go.Line(y = record.p_signal[:,0], x = np.array([i/atr.fs for i in range(len(record.p_signal[:,0]))])))
    else:
        fig.add_trace(go.Line(np.array([0,0,0,0,0,0,0,0,0,0])))
                      
    fig.update_xaxes(title_text='Электрокардиограмма (время, с)', range=(0, atr.fs*2/atr.fs))
    fig.update_yaxes(title_text='mV')   
    
    ecg_dang_value_max = 85
    ecg_dang_value_min = 45
    a = atr.sample[np.where((np.diff(atr.sample)/atr.fs*60 > ecg_dang_value_max) | (np.diff(atr.sample)/atr.fs*60 < ecg_dang_value_min))[0]]
    
    if len(a) > 0:
        fig.add_trace(go.Scatter(
            x=(a)/atr.fs, y=np.array([10 for i in a]),
            mode='markers',
            marker_color='rgba(152, 0, 0, .8)'
        ))   
        dang = 'присутствует'
    else:
        dang = 'отсутствует'
    fig.update_layout(showlegend=False)
    return fig, \
'Минимальный RR {}, мс'.format(round(np.min(np.diff(atr.sample))/atr.fs,3)), \
'Средний RR {}, мс'.format(round(np.mean(np.diff(atr.sample)/atr.fs), 3)), \
'Максимальный RR {}, мс'.format(round(np.max(np.diff(atr.sample))/atr.fs,3)), \
'ЧСС {}, уд/м'.format(round(len(atr.sample)/len(record.p_signal)*atr.fs*60)), \
'Состояние опасности (электрокардиограмма) {}'.format(dang)

# Update the index
@app.callback(
    Output('page-content', 'children'),
    Input('url', 'pathname'))
def display_page(pathname):
    if pathname == '/page-1':  
        return page_1_layout
    elif pathname == '/page-2':
        return page_2_layout
    else:
        return index_page
    
if __name__ == '__main__':
    webbrowser.open('http://localhost:3031/', new=2)
    app.run_server(host='localhost', port='3031', debug=False)
    

Dash is running on http://localhost:3031/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:3031/ (Press CTRL+C to quit)
